In [1]:
import pandas as pd

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import altair as alt
from altair import datum

In [2]:
def review_data_import_clean(product_name, verified_purchase = 1):
    df_review = pd.read_csv('data/{}.csv'.format(product_name))
    df_review = df_review[df_review['verified_purchase'] == verified_purchase]
    df_review['review_date_key'] = pd.to_datetime(df_review['review_date'])
    df_review_weekly = df_review.groupby(pd.Grouper(key='review_date_key', freq='W-SUN', label='left', closed='left'))['review_rating']\
                        .agg(['mean', 'count'])\
                        .reset_index()
    df_review_weekly.rename(columns={'mean':'average_rating', 'count':'review_count'}, inplace=True)
    
    return df_review_weekly

In [3]:
def google_data_import_clean(product_name):
    df_gg = pd.read_excel('data/Google Trend.xlsx', sheet_name = product_name)
    df_gg['week_key'] = pd.to_datetime(df_gg['Week'])
    
    df_gg.rename(columns = {df_gg.columns[1]:'trend_index'}, inplace = True)
    df_gg_filter = df_gg[df_gg['trend_index'].cumsum() > 0]
    
    return df_gg_filter

In [4]:
def data_import_clean(product_name, price, veri):
    
    df_review_clean = review_data_import_clean(product_name)
    df_gg_clean = google_data_import_clean(product_name)
    
    df = df_review_clean.merge(df_gg_clean, left_on = 'review_date_key', right_on='week_key', how='right')
    df['product_name'] = product_name
    df['price'] = price
    df = df.sort_values('week_key').reset_index().drop('index', axis=1)
    
    return df

In [5]:
# def visualize_review_google_trend(df):
    
#     fig = plt.figure(figsize=(20,10))
#     ax = fig.add_subplot(111)

#     lns1 = ax.plot(df['week_key'], df['review_count'], '-b', label = 'Amazon Review Count')
#     ax2 = ax.twinx()
#     lns2 = ax2.plot(df['week_key'], df['trend_index'],'-r', label = 'Google Trend Index')

#     lns = lns1+lns2
#     labs = [l.get_label() for l in lns]
#     ax.legend(lns, labs, loc=0, fontsize=16)

#     ax.set_xlabel("Week")
#     ax.set_ylabel("Review Count")
#     ax2.set_ylabel("Trend Index")

#     fig.tight_layout()
#     plt.show()

In [6]:
# def visualize_correlation(df):
#     corr_lst = [(i, df['review_count'].corr(df['trend_index'].shift(i))) for i in range(len(df)//2)]
#     plt.bar([i[0] for i in corr_lst], [i[1] for i in corr_lst])
#     print(max(corr_lst,key=lambda item:item[1]))

In [7]:
def calculate_correlation_review_count(df):
    corr_lst = [(i, df['review_count'].corr(df['trend_index'].shift(i))) for i in range(len(df)//2)]
    df_corr = pd.DataFrame(corr_lst, columns=['lag', 'corr'])
    df_corr['product_name'] = df['product_name'][0]
    df_corr['price'] = df['price'][0]
    return df_corr

In [8]:
def calculate_correlation_review_rating(df):
    corr_lst = [(i, df['average_rating'].corr(df['trend_index'].shift(i))) for i in range(len(df)//2)]
    df_corr = pd.DataFrame(corr_lst, columns=['lag', 'corr'])
    df_corr['product_name'] = df['product_name'][0]
    df_corr['price'] = df['price'][0]
    return df_corr

In [9]:
product_list = ['AirPods Pro', 'Elite 75t', 'Free', 'Galaxy Buds Plus', 'NC700', 'Soundcore Liberty 2 Pro', 'Soundcore Liberty Air 2', 'Soundcore Life Q20', 'WF-1000XM3']
price_list = [249.0, 179.0, 49.0, 149.0, 399.0, 149.0, 100.0, 59.0, 198.0]

In [10]:
df_corr_review_count = pd.DataFrame(columns=['lag', 'corr', 'product_name', 'price'])

for product, price in zip(*[product_list, price_list]):
    df = data_import_clean(product, price, 1)
    df_corr_review_count = pd.concat([df_corr_review_count, calculate_correlation_review_count(df)])
    
df_corr_review_count['segment'] = pd.cut(df_corr_review_count['price'], bins=[0,100,150,200,1000], labels=['Below 100', '100 to 150', '150 to 200', 'Above 200'], right=False)

In [11]:
df_corr_review_count

,lag,corr,product_name,price,segment
0,0,-0.316013,AirPods Pro,249.0,Above 200
1,1,0.074203,AirPods Pro,249.0,Above 200
2,2,0.376190,AirPods Pro,249.0,Above 200
3,3,0.412419,AirPods Pro,249.0,Above 200
4,4,0.427761,AirPods Pro,249.0,Above 200
...,...,...,...,...,...
17,17,-0.172343,WF-1000XM3,198.0,150 to 200
18,18,-0.033549,WF-1000XM3,198.0,150 to 200
19,19,-0.147340,WF-1000XM3,198.0,150 to 200
20,20,-0.421058,WF-1000XM3,198.0,150 to 200


A good correlation can be found for more expensive products after shifting Google Trend data by 6 to 9 weeks

In [12]:
alt.Chart(df_corr_review_count).mark_bar(size=10).encode(
    x=alt.X('lag:O'),
    y=alt.Y('corr:Q'),
    facet=alt.Facet(
        'product_name',
        columns=1,
        sort=['price']
    )
).resolve_scale(x='independent')

alt.Chart(...)

In [13]:
circle = alt.Chart(df_corr_review_count).mark_circle(
    size = 50
).transform_window(
        sort=[alt.SortField("corr", order="descending")], 
        rank="rank(corr)",
        groupby=['segment','product_name']
).transform_filter(
    alt.datum.rank == 1
).encode(
        x=alt.X("lag:O"), 
        y=alt.Y("corr:Q"),
        color='product_name:N',
)

text = alt.Chart(df_corr_review_count).mark_text(
    dy=-10, 
    dx=-15,
    color="red"
).transform_window(
        sort=[alt.SortField("corr", order="descending")], 
        rank="rank(corr)",
        groupby=['segment','product_name']
).transform_filter(
    alt.datum.rank == 1
).encode(
        x=alt.X("lag:O"), 
        y=alt.Y("corr:Q"),
        text=alt.Text("product_name:N"),
)

(circle + text).facet(
    column=alt.Column('segment', sort=['Below 100', '100 to 150', '150 to 200', 'Above 200'])
).properties(
    title='Best Correlation Found for Each Product in Different Price Segments')

alt.FacetChart(...)

Take Soundcore Liberty Air 2 as an example to visualize the correlation between normalized weekly review count and shifted weekly Google trend index

In [14]:
df_sc_air_2 = data_import_clean('Soundcore Liberty Air 2', 100)
df_sc_air_2['trend_index_shifted'] = df_sc_air_2['trend_index'].shift(6)
df_sc_air_2.dropna(subset=['review_count', 'trend_index_shifted'], how='any', inplace=True)
df_sc_air_2['review_count_normalized'] = (df_sc_air_2['review_count']/df_sc_air_2['review_count'].max()*100).astype(int)

alt.Chart(df_sc_air_2).mark_line().transform_fold(
    fold=['review_count_normalized', 'trend_index_shifted'], 
    as_=['variable', 'value']
).encode(
    x='week_key',
    y='value:Q',
    color='variable:N'
)

alt.Chart(...)

Another example of WF-1000XM3

In [15]:
df_wf_1000_xm3 = data_import_clean('WF-1000XM3', 100)
df_wf_1000_xm3['trend_index_shifted'] = df_wf_1000_xm3['trend_index'].shift(9)
df_wf_1000_xm3.dropna(subset=['review_count', 'trend_index_shifted'], how='any', inplace=True)
df_wf_1000_xm3['review_count_normalized'] = (df_wf_1000_xm3['review_count']/df_wf_1000_xm3['review_count'].max()*100).astype(int)

alt.Chart(df_wf_1000_xm3).mark_line().transform_fold(
    fold=['review_count_normalized', 'trend_index_shifted'], 
    as_=['variable', 'value']
).encode(
    x='week_key',
    y='value:Q',
    color='variable:N'
)

alt.Chart(...)

Example of Elite 75t

In [16]:
df_elite_75t = data_import_clean('Elite 75t', 100)
df_elite_75t['trend_index_shifted'] = df_elite_75t['trend_index'].shift(9)
df_elite_75t.dropna(subset=['review_count', 'trend_index_shifted'], how='any', inplace=True)
df_elite_75t['review_count_normalized'] = (df_elite_75t['review_count']/df_elite_75t['review_count'].max()*100).astype(int)

alt.Chart(df_elite_75t).mark_line().transform_fold(
    fold=['review_count_normalized', 'trend_index_shifted'], 
    as_=['variable', 'value']
).encode(
    x='week_key',
    y='value:Q',
    color='variable:N'
)

alt.Chart(...)

Example of NC700

In [17]:
df_nc_700 = data_import_clean('NC700', 100)
df_nc_700['trend_index_shifted'] = df_nc_700['trend_index'].shift(6)
df_nc_700.dropna(subset=['review_count', 'trend_index_shifted'], how='any', inplace=True)
df_nc_700['review_count_normalized'] = (df_nc_700['review_count']/df_nc_700['review_count'].max()*100).astype(int)

alt.Chart(df_nc_700).mark_line().transform_fold(
    fold=['review_count_normalized', 'trend_index_shifted'], 
    as_=['variable', 'value']
).encode(
    x='week_key',
    y='value:Q',
    color='variable:N'
)

alt.Chart(...)

Example of AirPods Pro

In [18]:
df_airpods_pro = data_import_clean('AirPods Pro', 100)
df_airpods_pro['trend_index_shifted'] = df_airpods_pro['trend_index'].shift(9)
df_airpods_pro.dropna(subset=['review_count', 'trend_index_shifted'], how='any', inplace=True)
df_airpods_pro['review_count_normalized'] = (df_airpods_pro['review_count']/df_airpods_pro['review_count'].max()*100).astype(int)

alt.Chart(df_airpods_pro).mark_line().transform_fold(
    fold=['review_count_normalized', 'trend_index_shifted'], 
    as_=['variable', 'value']
).encode(
    x='week_key',
    y='value:Q',
    color='variable:N'
)

alt.Chart(...)

On the other hand, there is no good correlation between review rating and Google Trend for any shifted periods.

In [19]:
df_corr_review_rating = pd.DataFrame(columns=['lag', 'corr', 'product_name', 'price'])

for product, price in zip(*[product_list, price_list]):
    df = data_import_clean(product, price)
    df_corr_review_rating = pd.concat([df_corr_review_rating, calculate_correlation_review_rating(df)])
    
df_corr_review_rating['segment'] = pd.cut(df_corr_review_rating['price'], bins=[0,100,150,200,1000], labels=['Below 100', '100 to 150', '150 to 200', 'Above 200'], right=False)

In [20]:
df_corr_review_rating

,lag,corr,product_name,price,segment
0,0,0.296331,AirPods Pro,249.0,Above 200
1,1,0.248517,AirPods Pro,249.0,Above 200
2,2,0.268443,AirPods Pro,249.0,Above 200
3,3,0.324300,AirPods Pro,249.0,Above 200
4,4,0.455515,AirPods Pro,249.0,Above 200
...,...,...,...,...,...
17,17,-0.029099,WF-1000XM3,198.0,150 to 200
18,18,-0.088879,WF-1000XM3,198.0,150 to 200
19,19,0.013955,WF-1000XM3,198.0,150 to 200
20,20,-0.143263,WF-1000XM3,198.0,150 to 200


In [21]:
alt.Chart(df_corr_review_rating).mark_bar(size=10).encode(
    x=alt.X('lag:O'),
    y=alt.Y('corr:Q'),
    facet=alt.Facet(
        'product_name',
        columns=1,
        sort=['price']
    )
).resolve_scale(x='independent')

alt.Chart(...)